## Setting Sex, Rce, Sex+Race, Age+Education as protected attribute

In [19]:
import torch
import pandas as pd
import numpy as np
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import os
import warnings
import traceback

# Suppress PyTorch warnings about weights_only
warnings.filterwarnings("ignore", category=FutureWarning, module="torch.serialization")

# Try to import tabulate, fallback to simple printing if not available
try:
    from tabulate import tabulate
    HAS_TABULATE = True
except ImportError:
    HAS_TABULATE = False
    print("tabulate library not found. Using simple table formatting.")

class FCNN(nn.Module):
    """Standard FCNN model for non-quantized models"""
    def __init__(self, input_dim, hidden_dim=64):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

def get_device():
    """Get the appropriate device (CUDA if available, else CPU)"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    return torch.device('cpu')

def load_model(model_path):
    """
    Load a PyTorch model with special handling for different model formats
    """
    try:
        device = get_device()
        
        # Check if this is a quantized model based on filename
        is_quantized = any(q in model_path for q in ['INT4', 'INT8', 'INT16'])
        
        if is_quantized:
            # print(f"Loading quantized model: {model_path}")
            try:
                # Load as state_dict first
                state_dict = torch.load(model_path, map_location='cpu')
                
                # Check if it's a state_dict or a model
                if isinstance(state_dict, dict):
                    # print("Loaded state dictionary for quantized model")
                    
                    # We'll recreate the original model structure and load these weights later
                    return state_dict, device, True, True  # Last True indicates it's a dict
                else:
                    # print("Loaded quantized model object")
                    if hasattr(state_dict, 'eval'):
                        state_dict.eval()
                    return state_dict, device, True, False  # False indicates it's not a dict
                
            except Exception as e:
                print(f"Error loading quantized model: {str(e)}")
                raise
        else:
            # Handle regular model loading
            # print(f"Loading regular model: {model_path}")
            state_dict = torch.load(model_path, map_location='cpu')
            
            # Get input dimension from fc1 weight matrix
            input_dim = state_dict['fc1.weight'].shape[1]
            hidden_dim = state_dict['fc1.weight'].shape[0]
            
            # Create model with correct dimensions
            model = FCNN(input_dim=input_dim, hidden_dim=hidden_dim)
            
            # Load state dict
            model.load_state_dict(state_dict)
            model.eval()
            
            # Move to appropriate device
            if device.type == 'cuda':
                try:
                    model = model.to(device)
                except RuntimeError as e:
                    print(f"Warning: Could not move model to CUDA. Using CPU instead. Error: {e}")
                    device = torch.device('cpu')
            
            return model, device, False, False  # Not quantized, not a dict
        
    except Exception as e:
        print(f"Error in load_model: {str(e)}")
        raise

def prepare_adult_dataset(expected_features=14, protected_attribute='sex'):
    """
    Prepare Adult dataset with proper formatting for fairness analysis
    """
    # Define column names
    column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 
                   'marital-status', 'occupation', 'relationship', 'race', 
                   'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 
                   'native-country', 'income']
    
    # Load data
    data_path = '../data/raw/adult.csv'
    if not os.path.exists(data_path):
        data_path = '../../data/raw/adult.csv'  # Try alternate path
    
    df = pd.read_csv(data_path, 
                     header=None,
                     names=column_names,
                     skipinitialspace=True)
    
    # Clean the data
    string_columns = df.select_dtypes(include=['object']).columns
    for col in string_columns:
        df[col] = df[col].str.strip()
    
    # Ensure income is properly encoded
    df['income'] = (df['income'].str.contains('>50K')).astype(int)
    
    # Create base numerical features
    numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain',
                         'capital-loss', 'hours-per-week']
                         
    # Create all protected attribute variations
    # 1. Sex (binary)
    df['sex_binary'] = (df['sex'] == 'Male').astype(int)
    
    # 2. Race (binary: White vs non-White)
    df['race_binary'] = (df['race'] == 'White').astype(int)
    
    # 3. Race + Sex (intersectional)
    df['race_sex'] = ((df['race'] == 'White') & (df['sex'] == 'Male')).astype(int)
    
    # 4. Age + Education (intersectional)
    df['higher_edu'] = (df['education-num'] >= 12).astype(int)
    df['older'] = (df['age'] >= 40).astype(int)
    df['age_edu'] = ((df['age'] >= 40) & (df['education-num'] >= 12)).astype(int)
    
    # Scale numerical features
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])
    
    # Determine which protected attribute to use
    if protected_attribute == 'sex':
        actual_attribute = 'sex_binary'
    elif protected_attribute == 'race':
        actual_attribute = 'race_binary'
    elif protected_attribute == 'race+sex':
        actual_attribute = 'race_sex'
    elif protected_attribute == 'age+education':
        actual_attribute = 'age_edu'
    else:
        raise ValueError(f"Unsupported protected attribute: {protected_attribute}")
    
    # To ensure consistency, include the protected attribute with its original name
    df[protected_attribute] = df[actual_attribute]
    
    # Start with numerical features plus the selected protected attribute
    base_features = numerical_features + [protected_attribute]
    final_df = df[base_features + ['income']]
    
    # If we need more features to match expected dimension
    if expected_features > len(base_features):
        # Add categorical features one at a time until we have enough
        categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                               'relationship', 'native-country']
        
        # Don't include race or sex again if they're part of our protected attribute
        if protected_attribute == 'race' or protected_attribute == 'race+sex':
            if 'race' in categorical_features:
                categorical_features.remove('race')
                
        if protected_attribute == 'sex' or protected_attribute == 'race+sex':
            if 'sex' in categorical_features:
                categorical_features.remove('sex')
        
        for cat_feature in categorical_features:
            if len(final_df.columns) - 1 >= expected_features:  # -1 for 'income'
                break
                
            # Add this categorical feature
            cat_encoded = pd.get_dummies(df[cat_feature], prefix=cat_feature)
            final_df = pd.concat([final_df.drop('income', axis=1), 
                                cat_encoded, 
                                final_df['income']], axis=1)
    
    # If we have too many features, select only the needed amount
    if len(final_df.columns) - 1 > expected_features:  # -1 for 'income'
        # Always ensure the protected attribute is included
        keep_cols = [protected_attribute]
        
        # Add other columns until we reach the expected number
        remaining_cols = [c for c in final_df.columns if c != protected_attribute and c != 'income']
        keep_cols.extend(remaining_cols[:expected_features - 1])
        
        # Add income back
        keep_cols.append('income')
        final_df = final_df[keep_cols]
    
    # Convert all columns to float32 for PyTorch compatibility
    final_df = final_df.astype('float32')
    
    return final_df

def compute_fairness_metrics(model_or_dict, device, dataset, protected_attribute, privileged_groups=None, 
                           is_quantized=False, is_dict=False, input_dim=None):
    """
    Compute fairness metrics with special handling for dictionary models
    """
    if privileged_groups is None:
        privileged_groups = [{protected_attribute: 1}]
    
    # Ensure all data is float32
    dataset = dataset.astype('float32')
    
    # Convert to AIF360 format
    aif_dataset = BinaryLabelDataset(
        df=dataset,
        label_names=['income'],
        protected_attribute_names=[protected_attribute],
        privileged_protected_attributes=[[1]]
    )
    
    # Prepare input features
    X = torch.FloatTensor(dataset.drop('income', axis=1).values)
    
    # Move to appropriate device
    if device.type == 'cuda':
        try:
            X = X.to(device)
        except RuntimeError as e:
            print(f"Warning: Could not move input to CUDA. Using CPU instead. Error: {e}")
            X = X.cpu()
            device = torch.device('cpu')
    
    # If model is a dict, we need to manually apply the operations
    if is_dict:
        # Recreate a model structure with the right dimensions
        if input_dim is None:
            input_dim = X.shape[1]  # Use input dimension from dataset
            
        # Create a model with the right structure
        model = FCNN(input_dim=input_dim, hidden_dim=64)
        
        # Try to load weights from the state dict
        try:
            # Check if the dict has expected keys
            if 'fc1.weight' in model_or_dict and 'fc2.weight' in model_or_dict:
                model.load_state_dict(model_or_dict)
            elif 'state_dict' in model_or_dict:
                # Some models save state dict under 'state_dict' key
                model.load_state_dict(model_or_dict['state_dict'])
            else:
                print("WARNING: Could not load state dict, using random weights")
                
            model.eval()
            model = model.to(device)
            model_or_dict = model  # Use the recreated model
            is_dict = False  # Not a dict anymore
        except Exception as e:
            print(f"Error loading weights from dict: {str(e)}")
            print("Using random predictions as fallback")
            is_dict = True  # Still a dict, will use random predictions
    
    # Process in batches
    batch_size = 1000
    predictions = []
    
    with torch.no_grad():
        for i in range(0, len(X), batch_size):
            batch = X[i:i + batch_size]
            
            try:
                # For dictionary models that couldn't be converted, use random predictions
                if is_dict:
                    pred = np.random.random(batch.size(0))
                # For regular models or successfully converted dict models
                else:
                    pred = model_or_dict(batch)
                    
                    # Ensure we have the right shape and move to CPU
                    pred = pred.squeeze().cpu().numpy()
                
                predictions.append(pred)
                
            except Exception as e:
                print(f"Error during batch prediction: {str(e)}")
                # Use random predictions as fallback
                pred = np.random.random(batch.size(0))
                predictions.append(pred)
    
    y_pred = (np.concatenate(predictions) > 0.5).astype(float)
    y_true = dataset['income'].values
    
    # Calculate fairlearn metrics
    dem_parity = demographic_parity_difference(
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=dataset[protected_attribute]
    )
    
    eq_odds = equalized_odds_difference(
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=dataset[protected_attribute]
    )
    
    # Create classified dataset with model predictions
    classified_dataset = aif_dataset.copy()
    classified_dataset.labels = y_pred.reshape(-1, 1)
    
    # Use classification_metric for all fairness calculations
    classification_metric = ClassificationMetric(
        aif_dataset,
        classified_dataset,
        unprivileged_groups=[{protected_attribute: 0}],
        privileged_groups=privileged_groups
    )
    
    # Return metrics based on predictions
    return {
        'disparate_impact': classification_metric.disparate_impact(),
        'statistical_parity_difference': classification_metric.statistical_parity_difference(),
        'demographic_parity_difference': dem_parity,
        'equalized_odds_difference': eq_odds,
        'average_odds_difference': classification_metric.average_odds_difference(),
        'equal_opportunity_difference': classification_metric.equal_opportunity_difference(),
        'theil_index': classification_metric.theil_index()
    }

def analyze_model_fairness(model_path, protected_attribute='sex', input_dim=None):
    """
    Main function to analyze model fairness
    """
    print(f"\nAnalyzing model: {os.path.basename(model_path)}")
    
    # Load model
    try:
        model_or_dict, device, is_quantized, is_dict = load_model(model_path)
        
        # Get appropriate input dimension
        if input_dim is None:
            # Try to infer from model
            if not is_dict:
                try:
                    input_dim = model_or_dict.fc1.weight.shape[1]
                except AttributeError:
                    pass
                
            # If quantized or couldn't get from model, try to infer from base model
            if input_dim is None and is_quantized:
                # Try to find base model
                base_model_path = None
                for suffix in ['_INT4.pth', '_INT8.pth', '_INT16.pth']:
                    if suffix in model_path:
                        potential_base = model_path.replace(suffix, '.pth')
                        if os.path.exists(potential_base):
                            base_model_path = potential_base
                            break
                
                if base_model_path:
                    try:
                        base_state_dict = torch.load(base_model_path, map_location='cpu')
                        input_dim = base_state_dict['fc1.weight'].shape[1]
                    except Exception as e:
                        input_dim = 14  # Default for Adult dataset
                else:
                    input_dim = 14  # Default for Adult dataset
                    
            if input_dim is None:
                input_dim = 14  # Default for Adult dataset
        
        # Prepare dataset with correct features
        dataset = prepare_adult_dataset(expected_features=input_dim, 
                                       protected_attribute=protected_attribute)
        
        # Compute fairness metrics
        fairness_metrics = compute_fairness_metrics(
            model_or_dict,
            device,
            dataset,
            protected_attribute=protected_attribute,
            privileged_groups=[{protected_attribute: 1}],
            is_quantized=is_quantized,
            is_dict=is_dict,
            input_dim=input_dim
        )
        
        return fairness_metrics
    except Exception as e:
        print(f"Error analyzing model: {str(e)}")
        return None

def format_model_name(path):
    """Format model name for display in tables"""
    name = os.path.basename(path)
    
    # Simplify model names for better display
    if 'fcnn_model_adult_income' in name:
        if 'fcnn_model_adult_income_pruned_' in name:
            bit = name.split('pruned_')[1].split('.')[0]
            return f'B_PRU{bit}'
        else:return 'Baseline'
    elif 'adv_fcnn_model_adult_sex' in name:
        if 'adv_fcnn_model_adult_sex_pruned_' in name:
            bit = name.split('pruned_')[1].split('.')[0]
            return f'A_PRU{bit}'
        else:return 'Adversarial'
    elif 'fair_demographic_parity_fcnn_model_adult_sex' in name:
        if 'fair_demographic_parity_fcnn_model_adult_sex_pruned_' in name:
            bit = name.split('pruned_')[1].split('.')[0]
            return f'F_PRU{bit}'
        else:return 'Fair_DP'
    elif 'adult_baseline_INT' in name:
        bit = name.split('INT')[1].split('.')[0]
        return f'B_INT{bit}'
    elif 'adult_adv_INT' in name:
        bit = name.split('INT')[1].split('.')[0]
        return f'A_INT{bit}'
    elif 'adult_fair_dp_INT' in name:
        bit = name.split('INT')[1].split('.')[0]
        return f'F_INT{bit}'
    elif 'fcnn_student_model_adult_income_' in name:
        bit = name.split('income_')[1].split('.')[0]
        return f'B_DIS{bit}'
    elif 'adult_adv_DIS' in name:
        bit = name.split('DIS')[1].split('.')[0]
        return f'A_DIS{bit}'
    elif 'adult_fair_dp_DIS' in name:
        bit = name.split('DIS')[1].split('.')[0]
        return f'F_DIS{bit}'
    
    return name

def create_comparison_table(results, metrics_to_show=None):
    """
    Create a side-by-side comparison table of fairness metrics across models
    """
    if not results:
        print("No results to display")
        return
    
    # Default metrics to show
    if metrics_to_show is None:
        metrics_to_show = [
            'disparate_impact',
            'statistical_parity_difference',
            'demographic_parity_difference',
            'equalized_odds_difference',
            'average_odds_difference',
            'equal_opportunity_difference',
            'theil_index'
        ]
    
    # Initialize table headers
    headers = ['Metric']
    
    # Add column for each model
    models = []
    for model_name in results.keys():
        # Get the first attribute (usually 'sex')
        for attr in results[model_name]:
            if results[model_name][attr]:  # If metrics exist
                models.append(model_name)
                headers.append(format_model_name(model_name))
                break
    
    # Create rows for each metric
    table_data = []
    for metric in metrics_to_show:
        row = [metric.replace('_', ' ').title()]
        
        for model_name in models:
            # Get first attribute (usually 'sex')
            attr = next(iter(results[model_name].keys()))
            
            if results[model_name][attr] and metric in results[model_name][attr]:
                value = results[model_name][attr][metric]
                # Format the value
                row.append(f"{value:.4f}")
            else:
                row.append("N/A")
        
        table_data.append(row)
    
    # Print the table
    print("\n===== FAIRNESS METRICS COMPARISON =====")
    
    # Use tabulate if available, otherwise use simple formatting
    if HAS_TABULATE:
        print(tabulate(table_data, headers=headers, tablefmt="grid"))
    else:
        # Simple table formatting
        header_str = "| " + " | ".join(headers) + " |"
        divider = "-" * len(header_str)
        print(divider)
        print(header_str)
        print(divider)
        
        for row in table_data:
            row_str = "| " + " | ".join(str(cell) for cell in row) + " |"
            print(row_str)
            
        print(divider)
    
    # Return a dataframe for further analysis if needed
    return pd.DataFrame(table_data, columns=headers).set_index(headers[0])

def run_comprehensive_analysis(model_paths, protected_attributes=['sex']):
    """
    Create comparative analysis across all models and attributes
    """
    results = {}
    
    # First analyze standard models to get input dimensions
    # This helps with quantized models later
    input_dims = {}
    
    # First pass to collect dimensions from standard models
    for model_path in model_paths:
        if os.path.exists(model_path):  # Skip non-existent models
            model_name = os.path.basename(model_path)
            if not any(q in model_path for q in ['INT4', 'INT8', 'INT16']):
                try:
                    state_dict = torch.load(model_path, map_location='cpu')
                    base_name = model_name.split('.')[0]  # Remove extension
                    input_dims[base_name] = state_dict['fc1.weight'].shape[1]
                except Exception as e:
                    pass
    
    # Now run full analysis with dimensions information
    for model_path in model_paths:
        if not os.path.exists(model_path):
            print(f"Skipping non-existent model: {model_path}")
            continue
            
        model_name = os.path.basename(model_path)
        results[model_name] = {}
        
        # For quantized models, try to find the input dimension
        input_dim = None
        if any(q in model_path for q in ['INT4', 'INT8', 'INT16']):
            # Extract base model name by removing the INT part
            for suffix in ['_INT4', '_INT8', '_INT16']:
                if suffix in model_name:
                    base_name = model_name.split(suffix)[0]
                    if base_name in input_dims:
                        input_dim = input_dims[base_name]
                        break
        
        for attr in protected_attributes:
            try:
                fairness_metrics = analyze_model_fairness(model_path, protected_attribute=attr, input_dim=input_dim)
                results[model_name][attr] = fairness_metrics
            except Exception as e:
                print(f"Error analyzing {model_name} with {attr}: {str(e)}")
    
    # Create comparison table
    comparison_df = create_comparison_table(results)
    
    return results, comparison_df

# Function to generate clean path that exists in the system
def get_valid_path(base_paths, filename):
    """Try multiple base paths to find one that exists with the given filename"""
    potential_paths = [
        os.path.join(path, filename) for path in base_paths
    ]
    
    for path in potential_paths:
        if os.path.exists(path):
            return path
    
    # If no path exists, return the first one (which will fail later with a clear error)
    return potential_paths[0]


In [22]:
# Base paths to search
base_paths = ['../models', '../../models']

# Group models by model type
grouped_models = {
    'baseline': [
        'baseline/fcnn_model_adult_income.pth',
        'quantized/adult_baseline_INT4.pth',
        'quantized/adult_baseline_INT8.pth',
        'quantized/adult_baseline_INT16.pth',
        'distillation/fcnn_student_model_adult_income_4.pth',
        'distillation/fcnn_student_model_adult_income_8.pth',
        'pruned/fcnn_model_adult_income_pruned_20pct.pth',
        'pruned/fcnn_model_adult_income_pruned_40pct.pth',
        'pruned/fcnn_model_adult_income_pruned_60pct.pth'
    ],
    'fair_dp': [
        'debiased/fair_demographic_parity_fcnn_model_adult_sex.pth',
        'quantized/adult_fair_dp_INT4.pth',
        'quantized/adult_fair_dp_INT8.pth',
        'quantized/adult_fair_dp_INT16.pth',
        'distillation/adult_fair_dp_DIS4.pth',
        'distillation/adult_fair_dp_DIS8.pth',
        'pruned/fair_demographic_parity_fcnn_model_adult_sex_pruned_20pct.pth',
        'pruned/fair_demographic_parity_fcnn_model_adult_sex_pruned_40pct.pth',
        'pruned/fair_demographic_parity_fcnn_model_adult_sex_pruned_60pct.pth'
    ],
    'adversarial': [
        'debiased/adv_fcnn_model_adult_sex.pth',
        'quantized/adult_adv_INT4.pth',
        'quantized/adult_adv_INT8.pth',
        'quantized/adult_adv_INT16.pth',
        'distillation/adult_adv_DIS4.pth',
        'distillation/adult_adv_DIS8.pth',
        'pruned/adv_fcnn_model_adult_sex_pruned_20pct.pth',
        'pruned/adv_fcnn_model_adult_sex_pruned_40pct.pth',
        'pruned/adv_fcnn_model_adult_sex_pruned_60pct.pth'
    ]
}

# Create paths with valid directories
model_paths = []
for group, filenames in grouped_models.items():
    for filename in filenames:
        model_paths.append(get_valid_path(base_paths, filename))

print("Starting analysis of all models...")
# Run analysis for a single protected attribute
results, comparison_df = run_comprehensive_analysis(model_paths, protected_attributes=['sex'])

# Display additional group-based comparisons
print("\n===== MODEL COMPARISON BY GROUP =====")
for group_name, group_files in grouped_models.items():
    group_results = {}
    for path in model_paths:
        basename = os.path.basename(path)
        for file in group_files:
            if file.split('/')[-1] == basename:
                if basename in results:
                    group_results[basename] = results[basename]
    
    if group_results:
        print(f"\n{group_name.upper()} GROUP COMPARISON:")
        create_comparison_table(group_results)

Starting analysis of all models...

Analyzing model: fcnn_model_adult_income.pth


C:\Users\Jasper\AppData\Local\Temp\ipykernel_41288\328194755.py:536: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location='cpu')



Analyzing model: adult_baseline_INT4.pth


C:\Users\Jasper\AppData\Local\Temp\ipykernel_41288\328194755.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location='cpu')



Analyzing model: adult_baseline_INT8.pth


...


Analyzing model: adv_fcnn_model_adult_sex_pruned_40pct.pth


C:\Users\Jasper\AppData\Local\Temp\ipykernel_41288\328194755.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location='cpu')



Analyzing model: adv_fcnn_model_adult_sex_pruned_60pct.pth


C:\Users\Jasper\AppData\Local\Temp\ipykernel_41288\328194755.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location='cpu')



===== FAIRNESS METRICS COMPARISON =====
+-------------------------------+------------+----------+----------+-----------+----------+----------+--------------+--------------+--------------+-----------+----------+----------+-----------+----------+----------+--------------+--------------+--------------+---------------+----------+----------+-----------+----------+----------+--------------+--------------+--------------+
| Metric                        |   Baseline |   B_INT4 |   B_INT8 |   B_INT16 |   B_DIS4 |   B_DIS8 |   B_PRU20pct |   B_PRU40pct |   B_PRU60pct |   Fair_DP |   F_INT4 |   F_INT8 |   F_INT16 |   F_DIS4 |   F_DIS8 |   F_PRU20pct |   F_PRU40pct |   F_PRU60pct |   Adversarial |   A_INT4 |   A_INT8 |   A_INT16 |   A_DIS4 |   A_DIS8 |   A_PRU20pct |   A_PRU40pct |   A_PRU60pct |
+===============================+============+==========+==========+===========+==========+==========+==============+==============+==============+===========+==========+==========+===========+==========